**Informações das atas do COPOM**

* Atas: https://www.bcb.gov.br/publicacoes/atascopom/cronologicos

    * De janeiro/1998 até junho/2016, as atas não possuem o arquivo PDF formatado na mesma estrutura que existe atualmente;
    * A partir de julho/2016 até os dias de hoje, as atas são divulgadas em um arquivo PDF.

---

* Minutes: https://www.bcb.gov.br/en/publications/copomminutes/cronologicos

    * O número de atas publicadas em inglês é diferente das atas publicadas em português. A primeira ata (nº 42) publicada em inglês é de 01/2000;
    * Desde o começo das suas publicações em inglês, os arquivos já possuiam um arquivo PDF.

---

**Estrutura do texto em português - atual**

* A) Atualização da conjuntura econômica e do cenário básico do Copom1
* B) Riscos em torno do cenário básico para a inflação
* C) Discussão sobre a condução da política monetária
* D) Decisão de política monetária

---

**Estrutura do texto em inglês - atual**
   
* A) Update of economic outlook and Copom’s baseline scenario1
* B) Scenarios and risk analysis
* C) Discussion about the conduct of monetary policy
* D) Monetary policy decision

## Informações auxiliares para rodar o script 'atas_pdf_selenium'

In [4]:
import requests
import re
import json

* Para rodar o script que faz o download dos arquivos pdfs das atas, eu preciso adicionar a data de publicação da ata na URL -> https://www.bcb.gov.br/en/publications/copomminutes/{date_minute};
* Essas datas são os valores do dicionário ('dict_ata').


In [5]:
# Dados do request da página do BC -> 'Inspecionar' -> 'Network' -> 'Fetch/XHR' -> 'Atualiza a página' -> endpoint 'ultimas?quantidade=1000&filtro' 
url = "https://www.bcb.gov.br/api/servico/sitebcb/copomminutes/ultimas"

querystring = {"quantidade":"1000","filtro":""}

payload = ""
headers = {"sec-ch-ua": "^\^Chromium^^;v=^\^116^^, ^\^Not"}

r = requests.request("GET", url, data=payload, headers=headers, params=querystring)
data = r.json()

# Lista com os títulos das atas
lst_titulo = [item['Titulo'] for item in data['conteudo']]

# Removendo da lista o título do pdf 'Changes in Copom meetings'
string_a_remover = 'Changes in Copom meetings'

if string_a_remover in lst_titulo:
    lst_titulo.remove(string_a_remover)
else:
    print(f'A string {string_a_remover} não está na lista.')

# Selecionando apenas o número da ata do título
# Observação: o título da ata 210 ('210 Copom Minutes') é o único que não tem o 'th' após o número da ata -> '210 th'
# Esse diferença estava zoando o código, porque eu estava fazendo o regex para selecionar o número antes de 'st', 'nd', 'rd' e 'th'
# Quando eu percebi essa discrepância no título da ata 210, eu consegui arrumar o regex para selecionar os números inicias de cada string
lst_titulo = [re.findall(r'^\d+', item)[0] if re.findall(r'^\d+', item) else None for item in lst_titulo]


# Lista com as datas das publicações das atas
lst_data_pub = [item['DataReferencia'] for item in data['conteudo']]

# Removendo da lista a data que corresponde ao pdf 'Changes in Copom meetings'
data_a_remover = '2000-01-19T02:00:00Z'

if data_a_remover in lst_data_pub:
    lst_data_pub.remove(data_a_remover)
else:
    print(f'A string {data_a_remover} não está na lista.')

# Usando regex para encontrar todas as datas no formato 'yyyy-mm-dd' na lista
padrao_data = r'\d{4}-\d{2}-\d{2}'
lst_data_pub = re.findall(padrao_data, ' '.join(lst_data_pub))

# Removendo os traços '-' de cada data
lst_data_pub = [data.replace('-', '') for data in lst_data_pub]

# Revertendo a ordem das datas para 'ddmmyyyy'
lst_data_pub = [data[6:] + data[4:6] + data[:4] for data in lst_data_pub]


# Criando um dicionário que contém todas as atas disponíveis
dict_ata = dict(zip(lst_titulo, lst_data_pub))
dict_ata 

{'257': '20092023',
 '256': '02082023',
 '255': '21062023',
 '254': '03052023',
 '253': '22032023',
 '252': '01022023',
 '251': '07122022',
 '250': '26102022',
 '249': '21092022',
 '248': '03082022',
 '247': '15062022',
 '246': '04052022',
 '245': '16032022',
 '244': '02022022',
 '243': '08122021',
 '242': '27102021',
 '241': '22092021',
 '240': '04082021',
 '239': '16062021',
 '238': '05052021',
 '237': '17032021',
 '236': '20012021',
 '235': '09122020',
 '234': '28102020',
 '233': '16092020',
 '232': '05082020',
 '231': '17062020',
 '230': '06052020',
 '229': '18032020',
 '228': '05022020',
 '227': '11122019',
 '226': '30102019',
 '225': '18092019',
 '224': '31072019',
 '223': '19062019',
 '222': '08052019',
 '221': '21032019',
 '220': '06022019',
 '219': '12122018',
 '218': '31102018',
 '217': '19092018',
 '216': '01082018',
 '215': '20062018',
 '214': '16052018',
 '213': '27032018',
 '212': '15022018',
 '211': '12122017',
 '210': '31102017',
 '209': '13092017',
 '208': '01072017',


* Eu não quero fazer o download de todos os arquivos PDF ao mesmo tempo. Para fazer os download das atas em partes, eu tenho que descobrir a posição da ata na lista 'lst_data_pub';
* Eu criei o 'dict_ata' para ajudar na visualização para saber qual é a data de publicação correspondente com o nº da ata;
* A 'lst_data_pub' contém apenas as datas de publicações;
* Na 'lst_data_pub', o primeiro valor 'lst_data_pub[0]' sempre é a ata mais atual.
    

In [14]:
lst_data_pub[26:27]

['17062020']

* O número máximo que eu consigo iterar é o 'len(lst_data_pub)' que é igual ao 'len(dict_ata)'.

In [5]:
# Número total de atas disponíveis
print(f'O número total é de {len(dict_ata)} atas.')
print(f'O número total é de {len(lst_data_pub)} atas.')

O número total é de 216 atas.
O número total é de 216 atas.


* Descobrindo qual é a posição na lista de uma ata específica:

In [77]:
# Número da ata que você deseja encontrar
numero_da_ata = '200'

# Verificar a posição da ata no dicionário
posicao = None
for i, chave in enumerate(dict_ata.keys(), start=1):
    if chave == numero_da_ata:
        posicao = i
        break

if posicao is not None:
    print(f"A ata nº {numero_da_ata} está na posição {posicao - 1} do dicionário.")
else:
    print(f"A ata nº {numero_da_ata} não foi encontrada no dicionário.")

A ata nº 200 está na posição 57 do dicionário.


* Agora que sabemos qual é a posição de uma ata específica eu posso criar um slice na lista;
* Exemplo: ata nº 42 (primeira ata publicada pelo BC, ela está na última posição da lista) até a ata nº 50

In [78]:
# Lista com as datas de publicações de todas as atas
lst_data_pub[46:57]

['12122017',
 '31102017',
 '13092017',
 '01072017',
 '06052017',
 '18042017',
 '02022017',
 '17012017',
 '06112016',
 '25102016',
 '31082016']

* Se as datas do slicing estiverem corretas. Agora, basta ir para o script 'atas_pdf_selenium' e colocar esses números nos parâmetros 'pos_ata_inicial' e 'pos_ata_final' da função 'download_arquivo'.

## Renomeando os nomes dos arquivos das atas do COPOM

In [1]:
import glob
import os

In [16]:
path_download = r"C:\Users\vitor\projetos_python\python_b3\historico-arquivos\minutes-pdf"

arquivos = glob.glob(os.path.join(path_download, '*.pdf'))

for arquivo in arquivos:
    # Selecionando apenas os nomes do arquivo
    nome_do_arquivo = os.path.basename(arquivo)

    # Padrão do nome: 'min200042-copom20000110-42nd_copom_minutes.pdf'
    if nome_do_arquivo.startswith('min2000'):
        # Selecionando apenas o nº da ata que vem antes de rt|nd|rd|th
        padrao = r'(\d+)(?:rt|nd|rd|th)'
        correspondencias = re.findall(padrao, nome_do_arquivo)
        num_ata = correspondencias[0]
        novo_nome = f"minutes_{num_ata}.pdf"
        
        # Renomeando o nome dos arquivos
        caminho_atual = os.path.join(path_download, nome_do_arquivo)
        novo_caminho = os.path.join(path_download, novo_nome)
        os.rename(caminho_atual, novo_caminho)

    # Padrão do nome: 'min20190731224.pdf'
    if nome_do_arquivo.startswith('min2019'):
        # Selecionando apenas o nº da ata (três digitos que vem antes de um ponto final)
        padrao = r'\d{3}(?=\.)'
        correspondencias = re.findall(padrao, nome_do_arquivo)
        num_ata = correspondencias[0]
        novo_nome = f"minutes_{num_ata}.pdf"
        
        # Renomeando o nome dos arquivos
        caminho_atual = os.path.join(path_download, nome_do_arquivo)
        novo_caminho = os.path.join(path_download, novo_nome)
        os.rename(caminho_atual, novo_caminho)

    # Padrão do nome: 'Copom Minutes 230.pdf'
    if nome_do_arquivo.startswith('Copom Minutes'):
        # Substituindo espaços em branco por underscores, convertendo para letras minúsculas e retirando o 'copom_'
        novo_nome = nome_do_arquivo.replace(' ', '_').lower().replace('copom_', '')

        # Renomeando o nome dos arquivos
        caminho_atual = os.path.join(path_download, nome_do_arquivo)
        novo_caminho = os.path.join(path_download, novo_nome)
        os.rename(caminho_atual, novo_caminho)
    
    # Padrão do nome: 'Copom 229 Minutes.pdf'
    if nome_do_arquivo.startswith('Copom 229'):
        # Substituindo espaços em branco por underscores, convertendo para letras minúsculas e retirando o 'copom_'
        novo_nome = nome_do_arquivo.replace(' ', '_').lower().replace('copom_', '')
        # Invertendo '229_minutes.pdf' p/ 'minutes_229.pdf'
        partes = novo_nome.split("_")
        novo_nome = partes[1].replace(".pdf", "") + "_" + partes[0] + ".pdf"

        # Renomeando o nome dos arquivos
        caminho_atual = os.path.join(path_download, nome_do_arquivo)
        novo_caminho = os.path.join(path_download, novo_nome)
        os.rename(caminho_atual, novo_caminho)

    # Padrão do nome: 'MINUTES227-min20191211227.pdf'
    if nome_do_arquivo.startswith('MINUTES227'):
        # Dividindo a string com base no traço (-) p/ pegar a primeira parte 'MINUTES227'
        partes = nome_do_arquivo.split('-')
        novo_nome = partes[0] + '.pdf'
        # Convertendo para letras minúsculas
        novo_nome = novo_nome.lower()
        # Adionando o score no meio da string -> 'minutes_227'
        novo_nome = re.sub(r'(\d+)', r'_\1', novo_nome)

        # Renomeando o nome dos arquivos
        caminho_atual = os.path.join(path_download, nome_do_arquivo)
        novo_caminho = os.path.join(path_download, novo_nome)
        os.rename(caminho_atual, novo_caminho)
    
    # Padrão do nome: 'MINUTES 226.pdf'
    if nome_do_arquivo.startswith('MINUTES'):
        # Substituindo espaços em branco por underscores e convertendo para letras minúsculas
        novo_nome = nome_do_arquivo.replace(' ', '_').lower()

        # Renomeando o nome dos arquivos
        caminho_atual = os.path.join(path_download, nome_do_arquivo)
        novo_caminho = os.path.join(path_download, novo_nome)
        os.rename(caminho_atual, novo_caminho)
        
        print(novo_nome)
       


In [83]:
# Padrão do nome: 'MINUTES227-min20191211227.pdf'
padrao = r'MINUTES\d+-min\d+\.pdf'
# Padrão do nome: 'MINUTES218-min20181031218'
padrao_2 = r'MIN2018\d+'
# Padrão do nome: 'MIN2017211th-COPOM20171205-211th_Copom_Minutes'
padrao_3 = r'MIN2017\d+'
# Padrão do nome: 'MIN2016203rd-COPOM20161206-203rd_Copom_Minutes'
padrao_4 = r'MIN2016\d+'

for arquivo in arquivos:
    nome_arquivo = os.path.basename(arquivo).lower()

    # Strings correpondentes com o padrão regex
    correspondencias = re.findall(padrao, nome_arquivo, re.IGNORECASE)
    correspondencias_2 = re.findall(padrao_2, nome_arquivo, re.IGNORECASE)
    correspondencias_3 = re.findall(padrao_3, nome_arquivo, re.IGNORECASE)
    correspondencias_4 = re.findall(padrao_4, nome_arquivo, re.IGNORECASE)

    if correspondencias:
        for correspondencia in correspondencias:
            # Separando a string no '-'
            partes = correspondencia.split('-')
            # Adicionando '.pdf' na string
            novo_nome = partes[0] + '.pdf'
            # Adicionando o underscore entre a palavra 'minutes' e o número da ata
            novo_nome = novo_nome.replace('minutes', 'minutes_')
            # Construindo o novo caminho
            novo_caminho = os.path.join(path_download, novo_nome)
            # Renomeando o arquivo
            os.rename(arquivo, novo_caminho)
    
    if correspondencias_2:
        for correspondencia in correspondencias_2:
            # Adicionando '.pdf' na string
            novo_nome = correspondencia + '.pdf'
            # Substituindo a palavra 'min2018' por 'minutes_'
            novo_nome = novo_nome.replace('min2018', 'minutes_')
            # Construindo o novo caminho
            novo_caminho = os.path.join(path_download, novo_nome)
            # Renomeando o arquivo
            os.rename(arquivo, novo_caminho)

    if correspondencias_3:
        for correspondencia in correspondencias_3:
            # Adicionando '.pdf' na string
            novo_nome = correspondencia + '.pdf'
            # Substituindo a palavra 'min2018' por 'minutes_'
            novo_nome = novo_nome.replace('min2017', 'minutes_')
            # Construindo o novo caminho
            novo_caminho = os.path.join(path_download, novo_nome)
            # Renomeando o arquivo
            os.rename(arquivo, novo_caminho)

    if correspondencias_4:
        for correspondencia in correspondencias_4:
            # Adicionando '.pdf' na string
            novo_nome = correspondencia + '.pdf'
            # Substituindo a palavra 'min2018' por 'minutes_'
            novo_nome = novo_nome.replace('min2016', 'minutes_')
            # Construindo o novo caminho
            novo_caminho = os.path.join(path_download, novo_nome)
            # Renomeando o arquivo
            os.rename(arquivo, novo_caminho)

In [84]:
path_download = r"C:\Users\vitor\projetos_python\python_b3\historico-arquivos\minutes-pdf"

arquivos = glob.glob(os.path.join(path_download, '*.pdf'))

for arquivo in arquivos:
    # Selecionando apenas os nomes do arquivo
    nome_do_arquivo = os.path.basename(arquivo)

    print(nome_do_arquivo)

minutes_200.pdf
minutes_201.pdf
minutes_202.pdf
minutes_203.pdf
minutes_204.pdf
minutes_205.pdf
minutes_206.pdf
minutes_207.pdf
minutes_208.pdf
minutes_209.pdf
minutes_210.pdf
minutes_211.pdf
minutes_212.pdf
minutes_213.pdf
minutes_214.pdf
minutes_215.pdf
minutes_216.pdf
minutes_217.pdf
minutes_218.pdf
minutes_219.pdf
minutes_220.pdf
minutes_221.pdf
minutes_222.pdf
minutes_223.pdf
minutes_224.pdf
minutes_225.pdf
minutes_226.pdf
minutes_227.pdf
minutes_228.pdf
minutes_229.pdf
minutes_230.pdf
minutes_231.pdf
minutes_232.pdf
minutes_233.pdf
minutes_234.pdf
minutes_235.pdf
minutes_236.pdf
minutes_237.pdf
minutes_238.pdf
minutes_239.pdf
minutes_240.pdf
minutes_241.pdf
minutes_242.pdf
minutes_243.pdf
minutes_244.pdf
minutes_245.pdf
minutes_246.pdf
minutes_247.pdf
minutes_248.pdf
minutes_249.pdf
minutes_250.pdf
minutes_251.pdf
minutes_252.pdf
minutes_253.pdf
minutes_254.pdf
minutes_255.pdf
minutes_256.pdf
minutes_257.pdf
minutes_42.pdf
minutes_43.pdf
minutes_44.pdf
minutes_45.pdf
minutes_46.p